In [18]:
import os
import glob

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

In [13]:
def load_clean_csv(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # '날짜 시간' 행의 위치를 찾아 그 전까지 스킵
    header_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("날짜 시간"):
            header_idx = i
            break

    if header_idx is None:
        raise ValueError(f"'날짜 시간' 행을 찾을 수 없습니다: {path}")

    # pandas로 테이블 부분만 읽기
    df = pd.read_csv(path, skiprows=header_idx)
    return df

In [12]:
data_dir = "data"
dataset = glob.glob(os.path.join(data_dir, "**", "*.csv"), recursive=True)

In [15]:
sample_path = dataset[0]
df = load_clean_csv(sample_path)

print("파일 경로:", sample_path)
print("데이터 shape:", df.shape)
print("컬럼명:", df.columns.tolist())
# print(df.head())

파일 경로: data\20250401\포인트3\VuSitu_라이브_리딩_2025-04-01_12-26-23_장치_위치.csv
데이터 shape: (66, 25)
컬럼명: ['날짜 시간', 'Actual Conductivity (µS/cm) (1066342)', 'Specific Conductivity (µS/cm) (1066342)', '염도 (PSU) (1066342)', '저항력 (Ω⋅cm) (1066342)', '밀도 (g/cm³) (1066342)', '총 용존 고형물 (TDS) (ppt) (1066342)', 'pH (pH) (1057024)', 'pH mV (mV) (1057024)', 'ORP (mV) (1057024)', '온도 (°C) (1068851)', '압력 (psi) (1064654)', '깊이 (m) (1064654)', '표면 고도 (m) (1064654)', 'RDO 농도 (mg/L) (954526)', 'RDO 포화 (%Sat) (954526)', '산소분압 (Torr) (954526)', 'Chlorophyll-a Fluorescence (RFU) (945787)', 'Chlorophyll-a 농도 (µg/L) (945787)', '외부 전압 (V) (1068851)', '대기압 (mbar) (1049665)', '온도 (°C) (1049665)', '위도 (°)', '경도 (°)', '표시됨']


In [16]:
data_num = len(dataset)
train_dataset, val_dataset, test_dataset = dataset[:int(data_num*0.8)], dataset[int(data_num*0.8):int(data_num*0.9)], dataset[int(data_num*0.9):]
print(len(train_dataset), len(val_dataset), len(test_dataset))

392 49 49


In [19]:
def make_sequences(df_point, input_days=7):
    X, y = [], []
    for i in range(len(df_point) - input_days):
        X.append(df_point.iloc[i:i+input_days].values)
        y.append(df_point.iloc[i+input_days]["chlorophyll"])  # 다음날 예측
    return np.array(X), np.array(y)

In [ ]:
class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        """
        input_size: 입력 feature 수
        hidden_size: LSTM hidden 차원
        num_layers: LSTM 층 수
        output_size: 예측값 차원
        """
        super(VanillaLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        x: [batch_size, seq_len, input_size]
        """
        out, _ = self.lstm(x)       # out: [batch, seq_len, hidden_size]
        out = out[:, -1, :]          # 마지막 시점 출력
        out = self.fc(out)           # [batch, output_size]
        return out